In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acovf
from scipy.signal import periodogram


fn="../prices.txt"
df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
prices = (df.values).T

nInst ,T = 50,750

## Time series differncing 

In [ ]:
t = 750
tx = np.linspace(1,t,t)
pdiff = np.zeros((nInst,t))
for i in range(nInst):
    current = prices[i,:t]
    previous = np.zeros((t,))
    previous[0] = 1e-8
    previous[1:] = current[:-1]
    delta = current-previous
    pdiff[i,:] = delta

In [ ]:

colors = np.random.rand(nInst, 3)
def plotSplit():
    f,ax = plt.subplots(figsize=(20,10))
    for i in [0]:
        ax.plot(tx[1:], pdiff[i][1:],alpha=0.3,color=colors[i])
        lastDiff = pdiff[i][-1]
        lastT = tx[-1]
        ax.text(lastT,lastDiff,str(i))
    plt.show()
plotSplit()

# ARIMA

In [ ]:
instrument = 40
day = 749

# Simulated example price series
instPrice = prices[instrument,:day]

# Step 1: Define the ARIMA model
# ARIMA(p,d,q): 
#   p = AR order
#   d = differencing order
#   q = MA order
model = ARIMA(instPrice, order=(6,1,2),seasonal_order=(1,1,1,7))  # This (1,1,1) is a common starting point

# Step 2: Fit the model
model_fit = model.fit()

# Step 3: Print summary (optional)
print(model_fit.summary())

# Step 4: Forecast future prices
forecast_steps = 1
forecast = model_fit.forecast(steps=forecast_steps)

# Step 5: Plot
groundTruth = prices[instrument,:day+forecast_steps]
plt.plot(np.arange(len(groundTruth)), groundTruth, label='Observed')
plt.plot(np.arange(len(instPrice), len(instPrice)+forecast_steps), forecast, label='Forecast')
plt.legend()
plt.title("ARIMA Forecast")
plt.xlabel("Time")
plt.ylabel("Price")
plt.grid(True)
plt.show()
predictedRegionTrue = prices[instrument,day:day+forecast_steps]
plt.plot(np.arange(len(forecast)),predictedRegionTrue,label="True")
plt.plot(np.arange(len(forecast)),forecast,label="Forecast")
plt.legend()
plt.plot()


## AutoCov FFT

In [ ]:
acov = acovf(instPrice, fft=False)  # Full autocovariance

def full_autocovariance(x):
    x = np.asarray(x)
    n = len(x)
    x_mean = np.mean(x)
    result = np.correlate(x - x_mean, x - x_mean, mode='full') / n
    return result[n-1:]  # keep only non-negative lags

customAutoCov = full_autocovariance(instPrice)


plt.plot(np.arange(len(acov)),acov)
plt.show()

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def remove_exponential_decay(x, y, min_clip=1e-10):
    """
    Fits y ≈ A * exp(-lambda * x) using nonlinear curve fitting,
    and divides out the exponential decay.

    Args:
        x (np.ndarray): 1D array of x values.
        y (np.ndarray): 1D array of y values.
        min_clip (float): minimum value to clip y to avoid instability.

    Returns:
        y_flat (np.ndarray): y with exponential decay divided out.
        decay (np.ndarray): fitted decay curve.
        params (tuple): (A, lambda)
    """
    y_clipped = np.clip(y, min_clip, None)

    # Define the exponential model
    def exp_decay(x, A, lam):
        return A * np.exp(-lam * x)

    # Initial guess: A ≈ max(y), lambda ≈ small positive number
    p0 = [np.max(y_clipped), 0.1]

    # Fit the curve
    popt, _ = curve_fit(exp_decay, x, y_clipped, p0=p0)

    A, lambda_ = popt
    decay = A * np.exp(-lambda_ * x) + 2
    y_flat = y / decay

    return y_flat, decay, (A, lambda_)

In [ ]:
import numpy as np

def compute_fft(x, sampling_rate=1.0):
    """
    Compute the FFT and corresponding frequencies of a 1D signal.

    Parameters:
        x (array-like): Input time series (1D array).
        sampling_rate (float): Number of samples per time unit (default = 1.0).

    Returns:
        freqs (ndarray): Frequencies (Hz or inverse time units).
        magnitude (ndarray): Magnitude of the FFT.
        phase (ndarray): Phase angle (radians).
    """
    x = np.asarray(x)
    N = len(x)
    
    fft_result = np.fft.fft(x)
    freqs = np.fft.fftfreq(N, d=1/sampling_rate)
    
    magnitude = np.abs(fft_result)
    phase = np.angle(fft_result)
    
    return freqs, magnitude, phase

plt.figure(figsize=(8, 4))
x = None
y = None
for i in range(50):
    MADiff = np.convolve(pdiff[i],np.full(100,1))
    freqs, mag, phase = compute_fft(MADiff, sampling_rate=1)
    cx = freqs[:len(freqs)//2]
    cy = mag[:len(mag)//2]
    cy_flat, fitted_decay, params = remove_exponential_decay(cx, cy)
    if x is None:
        x = cx
    if y is None:
        y = cy
    y += cy_flat
y/=50

# Plot magnitude spectrum
plt.title("FFT Magnitude Spectrum")
plt.plot(x,y)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.grid(True)
plt.show()